In [10]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_performance_graphs(csv_file="runtime_log.csv"):
    try:
        df = pd.read_csv(csv_file)
    except FileNotFoundError:
        print(f"Error: File '{csv_file}' not found. Please check the file path.")
        return
    except pd.errors.ParserError:
        print(f"Error: Failed to parse the CSV file '{csv_file}'. Check the file format.")
        return
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return

    # Check if necessary columns exist (excluding detection_time)
    required_columns = ["timestamp", "fps", "planning_time", "distance_to_goal",
                        "occupancy_percent", "traversability_score", "left_velocity", "right_velocity"]
    if not all(col in df.columns for col in required_columns):
        missing_cols = [col for col in required_columns if col not in df.columns]
        print(f"Missing required columns: {missing_cols}")
        return

    # Convert timestamp to numeric (handle errors)
    df["timestamp"] = pd.to_numeric(df["timestamp"], errors='coerce')

    # Drop rows with missing or invalid data in required columns
    df.dropna(subset=required_columns, inplace=True)

    if df.empty:
        print("No valid data to plot after cleaning.")
        return

    # Create subplots with Plotly
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=("FPS and Planning Time", "Velocities over Time",
                                        "Distance to Goal", "Occupancy and Traversability"))

    # Plot FPS and Planning Time with enhanced styling
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["fps"], mode='lines+markers',
                             name="FPS", line=dict(color="green", width=2), marker=dict(size=5)),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["planning_time"], mode='lines+markers',
                             name="Planning Time", line=dict(color="blue", width=2, dash='dash'), marker=dict(size=5)),
                  row=1, col=1)
    fig.update_yaxes(title_text="FPS / Planning Time (s)", row=1, col=1)

    # Plot Velocities over Time
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["left_velocity"], mode='lines+markers',
                             name="Left Velocity", line=dict(color="red", width=2), marker=dict(size=5)),
                  row=2, col=1)
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["right_velocity"], mode='lines+markers',
                             name="Right Velocity", line=dict(color="purple", width=2), marker=dict(size=5)),
                  row=2, col=1)
    fig.update_yaxes(title_text="Velocity", row=2, col=1)

    # Plot Distance to Goal
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["distance_to_goal"], mode='lines+markers',
                             name="Distance to Goal", line=dict(color="orange", width=2), marker=dict(size=5)),
                  row=3, col=1)
    fig.update_yaxes(title_text="Distance", row=3, col=1)

    # Plot Occupancy Percentage and Traversability Score (with scaling for traversability)
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["occupancy_percent"], mode='lines+markers',
                             name="Occupancy Percentage", line=dict(color="brown", width=2), marker=dict(size=5)),
                  row=4, col=1)
    # Scale traversability score
    traversability_scaled = (df["traversability_score"] - df["traversability_score"].min()) / (
            df["traversability_score"].max() - df["traversability_score"].min())
    fig.add_trace(go.Scatter(x=df["timestamp"], y=traversability_scaled, mode='lines+markers',
                             name="Traversability Score (Scaled)", line=dict(color="white", width=2), marker=dict(size=5)),
                  row=4, col=1)
    fig.update_yaxes(title_text="Percentage / Score", row=4, col=1)
    fig.update_xaxes(title_text="Time (s)", row=4, col=1)

    # Update layout for a more visually appealing style
    fig.update_layout(title_text="<b>Performance Graphs</b>", title_x=0.05,  # Centered title
                      height=1000, width=1000,
                      template="plotly_dark",  # Use a dark theme
                      legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),  # Horizontal legend
                      hovermode="x unified")  # Unified hover for all subplots
    fig.show()

# Call the function to plot the graphs
plot_performance_graphs("runtime_log.csv")